# Atmosphere
> Load and scale the maps of the noise from the atmosphere
- toc: True

* We have 8 independent realizations
* They scale with the number of telescopes, i.e. it doesn't make any difference if there are 1 or 3 channels in the same telescope, they do not help beat the atmosphere
* They were simulated for 10 days at 100% efficiency at Chile, 46.29% (SAT) and 37.23% (LAT) at Pole

In [ ]:
# default_exp atmosphere

In [ ]:
# export

import healpy as hp
import numpy as np
from pathlib import Path
import logging as log

from s4_design_sim_tool.core import (
    get_telescope,
    get_observing_efficiency,
    base_folder,
    simulations_observing_efficiency,
    mapmaking_naming,
    read_instrument_model,
)

In [ ]:
import toml

In [ ]:
config = toml.load("s4_reference_design.toml")

In [ ]:
log.basicConfig(level=log.INFO)

In [ ]:
# exports


def get_telecope_years(config, site, channel):
    """Compute the number of telescope/years in the CMB-S4 configuration

    config_telescopes : dict
        CMB-S4 telescopes configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    """
    telescope_years = 0
    for telescope_name, telescope_config in config["telescopes"][
        get_telescope(channel)
    ].items():
        if telescope_config["site"].lower() == site.lower():
            has_band = telescope_config.get(channel[:-1], 0) > 0
            telescope_years += has_band * telescope_config.get(
                "years", config["experiment"]["total_experiment_length_years"]
            )
    return telescope_years

In [ ]:
s4 = read_instrument_model()

In [ ]:
!tail -30 s4_reference_design.toml

## Compute telescope/years for the reference design

In [ ]:
for site in ["Pole", "Chile"]:
    for channel in s4:
        telescope_years = get_telecope_years(config, site, channel["band"])
        print(site, channel["band"], telescope_years)
        telescope = get_telescope(channel["band"])
        if site == "Chile":
            if telescope == "SAT":
                assert telescope_years == 0, "All SAT at Pole"
            elif channel["band"].startswith("ULFL"):
                assert telescope_years == 0, "No ULFL in Chile"              
            else:
                if "P" in channel["band"]:
                    assert telescope_years == 0, "Pole only LAT"
                else:                   
                    assert telescope_years == 14, "2 LAT each band"
        if site == "Pole":
            if telescope == "SAT":
                assert telescope_years == 14, "2 SAT telescopes each band"
            else:
                if "P" in channel["band"]:
                    assert telescope_years == 7, "1 LAT with all 4 bands"
                else:
                    assert telescope_years == 0, "Chile only LAT"
    print(30 * "=")

In [ ]:
# exports


def load_atmosphere(config, site, channel, realization=0, raw=False):
    """Load foreground maps for a channel

    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    realization : int
        Choose one of the available 8 realizations

    Returns
    -------
    output_map : numpy array
        Output map with all emissions combined, uses nan for missing pixels
    """

    telescope = get_telescope(channel)
    map_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_atmosphere_{telescope}_{channel}_{mapmaking_naming[telescope]}"
    )
    log.info(f"Reading {map_filename}")
    atmosphere_map = hp.read_map(
        Path(base_folder) / map_filename, (0, 1, 2), dtype=None, verbose=False
    )
    if raw:
        atmosphere_map[atmosphere_map == 0] = hp.UNSEEN
        return atmosphere_map

    atmosphere_map[atmosphere_map == hp.UNSEEN] = np.nan
    atmosphere_map[atmosphere_map == 0] = np.nan

    # input map is 10 days at 100% efficiency
    atmosphere_map *= np.sqrt(
        10
        * simulations_observing_efficiency[site.lower()][channel]
        / (
            365.25
            * get_observing_efficiency(
                config["experiment"]["observing_efficiency"], site, telescope, channel
            )
        )
    )
    atmosphere_map /= np.sqrt(get_telecope_years(config, site, channel))

    atmosphere_map[0] *= config["experiment"].get("atmosphere_scaling_T", 1)
    atmosphere_map[1:] *= config["experiment"].get("atmosphere_scaling_P", 1)

    return atmosphere_map

## Run on a channel and plot results

Available atmosphere maps

In [ ]:
filenames = !ls $base_folder/00000000/*atmo*

In [ ]:
import os.path
for f in map(os.path.basename, filenames):
    print(f)

In [ ]:
channel = "HFS1"
site = "Pole"
telescope = get_telescope(channel)

In [ ]:
input_map = hp.read_map(
    base_folder + "/00000000/" + \
    f"{site.lower()}_atmosphere_{telescope}_{channel}_telescope_all_time_all_filtered.fits.gz"
, (0,1,2))

In [ ]:
input_map[input_map == 0] = hp.UNSEEN

In [ ]:
output_map = load_atmosphere(config, site, channel, realization=0)

In [ ]:
np.nanmin(output_map), np.nanmax(output_map)

In [ ]:
output_map[np.isnan(output_map)] = hp.UNSEEN

In [ ]:
# Compare mask
np.testing.assert_allclose(input_map == hp.UNSEEN, output_map == hp.UNSEEN)

In [ ]:
channel

In [ ]:
get_observing_efficiency(
                config["experiment"]["observing_efficiency"], site, telescope, channel
            )

In [ ]:
np.testing.assert_allclose(input_map[input_map != hp.UNSEEN] * \
    np.sqrt(10 * simulations_observing_efficiency[site.lower()][channel] / (7 * 365.25 * 0.1359894704) / 2),
    output_map[output_map != hp.UNSEEN], rtol=1e-6)

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(output_map[0], min=-1e-5, max=1e-5, unit="K", title="Atmosphere I")

There are no systematics in the half-wave plate or bandpass mismatch, so almost all the atmosphere signal is rejected in polarization.

In [ ]:
hp.mollview(output_map[1], min=-1e-9, max=1e-9, unit="K", title="Atmosphere Q")

In [ ]:
hp.mollview(output_map[2], min=-1e-9, max=1e-9, unit="K", title="Atmosphere U")

## Test scaling

We want to be able to freely scale the atmosphere noise differently for Temperature and for Polarization

In [ ]:
config["experiment"]["atmosphere_scaling_T"] = 3

In [ ]:
config["experiment"]["atmosphere_scaling_P"] = 0.1

In [ ]:
output_map = load_atmosphere(config, site, channel, realization=0)

In [ ]:
output_map[np.isnan(output_map)] = hp.UNSEEN

In [ ]:
np.testing.assert_allclose(config["experiment"]["atmosphere_scaling_T"] * input_map[0][input_map[0] != hp.UNSEEN] * \
    np.sqrt(10 * simulations_observing_efficiency[site.lower()][channel] / (7 * 365.25 * 0.1359894704) / 2),
    output_map[0][output_map[0] != hp.UNSEEN], rtol=1e-6)

In [ ]:
np.testing.assert_allclose(config["experiment"]["atmosphere_scaling_P"] * input_map[1:][input_map[1:] != hp.UNSEEN] * \
    np.sqrt(10 * simulations_observing_efficiency[site.lower()][channel] / (7 * 365.25 * 0.1359894704) / 2),
    output_map[1:][output_map[1:] != hp.UNSEEN], rtol=1e-6)